In [1]:
import torch
from core.models import ResNetEncoder

device = "cuda:1"
encoder = ResNetEncoder(out_features=1024).to(device)

/home/heon/anaconda3/envs/camelyon/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/heon/anaconda3/envs/camelyon/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
import os
import glob

from torchvision.transforms import RandomCrop, GaussianBlur, RandomEqualize, RandomRotation, RandomAffine, Compose, ToTensor
from core.datasets import SimCLRDataSet
from matplotlib import pyplot as plt

images = glob.glob(os.path.join("/home/heon/heon_vast/camelyon17_patches_new/RUMC/malignant", "*"))
dataset = SimCLRDataSet(
    images, 
    augmentations=[
        GaussianBlur((3,3)), 
        RandomEqualize(), 
        RandomRotation(degrees=(0, 360)), 
        RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75))
    ],
    transforms=Compose([ToTensor()]),
    device=device
)

In [3]:
from core.trainer import SimCLRTrainer
from core.loss import SimCLRLoss
dataloder = torch.utils.data.DataLoader(dataset, batch_size=4)
xi, xj = next(iter(dataloder))
vectors_i = encoder(xi)
vectors_j = encoder(xj)
loss = SimCLRLoss(device=device)
loss(vectors_i, vectors_j)

tensor(2.0694, device='cuda:1', grad_fn=<NllLossBackward0>)

In [4]:
from core.metrics import AverageMeter
optimizer = torch.optim.Adam(encoder.parameters())
trainer = SimCLRTrainer(encoder, optimizer, loss, device=device)
encoder.to(device)
loss_meter: AverageMeter = trainer.run_epoch(dataloder, phase="train", epoch=1)

 |################################| train | EPOCH 1: [195/195] | eta:1 | total_loss: 1.9373  
